# Week 3 - Getting the data

In [14]:
import pandas as pd
import numpy as np

In [15]:
from bs4 import BeautifulSoup
import requests # library to handle requests

## Fill the dataframe

In [16]:
soup = BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text, 'html5lib')
table = soup.table
table_rows = table.find_all('tr')
table_header = table.find_all('th')
table_header
# Get all the headings of Lists
headings = []
for td in table.find_all("th"):
    # remove any newlines and extra spaces from left and right
    headings.append(td.text.replace('\n', '').strip())
l = []
# Note: I leave the original name 'Postcode'
# Get rows
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.replace('\n', '').strip() for tr in td]
    l.append(row)
df = pd.DataFrame(l, columns=headings)
print(df.shape[0])
# Cleaning data
# Drop null rows
indexNames = df[df['Postcode'].isnull() == True].index
df.drop(indexNames , inplace=True)
# Drop borough not assigned
indexNames = df[df['Borough'] == 'Not assigned'].index
df.drop(indexNames , inplace=True)
print(df.shape[0])
df.head()


288
210


,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


## Group, combine and rename
### A note about the example. When I read the Wikipedia page Regent Park was not present in the table...

In [17]:
#dfg: data frame grouped
dfg = df.groupby(['Postcode', 'Borough'], sort=False).apply(lambda x: ', '.join(x.Neighbourhood)).to_frame()
dfg.reset_index(inplace=True)
dfg.rename(columns={'Postcode':'PostalCode', dfg.columns[2]:'Neighborhood'}, inplace=True)
dfg.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


## Assign to Neighborhood the same name as the Brough in case of Not assigned
### (Borough Queen's Park seemed the only one Borough with Not assigend Neighborood)

In [18]:
dfg['Neighborhood'] = np.where((dfg.Neighborhood == 'Not assigned'), dfg['Borough'], dfg.Neighborhood)
dfg.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


## Shape

In [19]:
dfg.shape

(103, 3)